## Create dictionary of movie titles using IMDB data

Source: ftp://ftp.fu-berlin.de/pub/misc/movies/database/



**NOTE: We don't use the IMDB gazetteers in our analysis because of LICENSE issues in using IMDB data dumps. SEE: http://www.imdb.com/help/show_leaf?usedatasoftware and http://www.imdb.com/interfaces**

In [1]:
%%bash
cd data/extra_gazetteers/
wget ftp://ftp.fu-berlin.de/pub/misc/movies/database/movies.list.gz
zcat movies.list.gz | cut -f1 -d'(' | sed -s 's/\"//g' | sort | uniq > all_movie_titles.txt
echo $(wc -l all_movie_titles.txt)
cp all_movie_titles.txt ../cleaned/custom_lexicons/

1162093 all_movie_titles.txt


--2016-10-01 21:57:12--  ftp://ftp.fu-berlin.de/pub/misc/movies/database/movies.list.gz
           => ‘movies.list.gz.2’
Resolving ftp.fu-berlin.de (ftp.fu-berlin.de)... 130.133.3.130
Connecting to ftp.fu-berlin.de (ftp.fu-berlin.de)|130.133.3.130|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/misc/movies/database ... done.
==> SIZE movies.list.gz ... 39295854
==> PASV ... done.    ==> RETR movies.list.gz ... done.
Length: 39295854 (37M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  201K 3m11s
    50K .......... .......... .......... .......... ..........  0%  403K 2m23s
   100K .......... .......... .......... .......... ..........  0% 11.0M 96s
   150K .......... .......... .......... .......... ..........  0% 10.9M 73s
   200K .......... .......... .......... .......... ..........  0%  420K 76s
   250K .......... .......... .......... .......... .........

In [2]:
! wc -l data/cleaned/custom_lexicons/all_movie_titles.txt

1162093 data/cleaned/custom_lexicons/all_movie_titles.txt


## Music artists database

Source: http://data.discogs.com/

In [3]:
%%bash
cd data/extra_gazetteers/
wget http://discogs-data.s3-us-west-2.amazonaws.com/data/discogs_20160901_artists.xml.gz

--2016-10-01 22:04:16--  http://discogs-data.s3-us-west-2.amazonaws.com/data/discogs_20160901_artists.xml.gz
Resolving discogs-data.s3-us-west-2.amazonaws.com (discogs-data.s3-us-west-2.amazonaws.com)... 54.231.168.177
Connecting to discogs-data.s3-us-west-2.amazonaws.com (discogs-data.s3-us-west-2.amazonaws.com)|54.231.168.177|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184659053 (176M) [text/xml]
Saving to: ‘discogs_20160901_artists.xml.gz’

     0K .......... .......... .......... .......... ..........  0%  453K 6m38s
    50K .......... .......... .......... .......... ..........  0%  896K 5m0s
   100K .......... .......... .......... .......... ..........  0%  925K 4m25s
   150K .......... .......... .......... .......... ..........  0% 11.7M 3m22s
   200K .......... .......... .......... .......... ..........  0%  973K 3m19s
   250K .......... .......... .......... .......... ..........  0% 11.6M 2m48s
   300K .......... .......... .......... .........

In [1]:
from bs4 import BeautifulSoup
import gzip
import codecs

In [2]:
with gzip.open("data/extra_gazetteers/discogs_20160901_artists.xml.gz") as fp,\
     codecs.open("data/extra_gazetteers/musicartist_names.txt", "wb+", "utf-8") as fp1,\
     codecs.open("data/extra_gazetteers/musicartist_namevariants.txt", "wb+", "utf-8") as fp2:
    i = 0
    for line in fp:
        if i == 0:
            i += 1
            continue
        line = line.strip()
        i += 1
        try:
            p = BeautifulSoup(line, "lxml-xml")
            artist_names = p.select("artist > name, aliases > name, groups > name")
            artist_name_variants = p.select("namevariations > name")
            for k in artist_names:
                print >> fp1, unicode(k.text)
            for k in artist_name_variants:
                print >> fp2, unicode(k.text)
        except:
            print "i=%s" % i
    print "i=%s" % i

/home/entity/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.facebook.com/victorromeo25&#13;" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/entity/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://twitter.com/parrismitchell1&#13;" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/entity/

i=5161005


In [3]:
! sort data/extra_gazetteers/musicartist_names.txt | uniq > data/extra_gazetteers/musicartist_names.unique.txt
! sort data/extra_gazetteers/musicartist_namevariants.txt | uniq > data/extra_gazetteers/musicartist_namevariants.unique.txt

In [4]:
! cp data/extra_gazetteers/*.unique.txt data/cleaned/custom_lexicons/